# Unbalanced Reaction Changes
    ## For each reaction alter if needed: stoich, reactants/products, metabolites, formulas, and charge
# Checked before and after
    ### Difference between biomass < .0001
    ### Media changes
    ### Gene essentiallity predictions
    ### flux through reaction difference 

## Extra goal of adding pKa to model 


In [1]:
import cobra
import copy
import os
from os.path import join
from cobra import Model, Reaction, Metabolite
from __future__ import print_function
import json
from pprint import pprint
#!/usr/bin/env python3
import fileinput
import re
from cobra.test import create_test_model
from cobra.flux_analysis import sample
from cobra.medium import minimal_medium
from cobra.flux_analysis import flux_variability_analysis
import pandas
from time import time
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

# %matplotlib inline
# import plot_helper

import cobra.test
from cobra import Reaction, Metabolite, Model
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba



import logging
from operator import attrgetter

import pandas as pd
from numpy import zeros
from six import iteritems, print_
from six.moves import zip_longest
from tabulate import tabulate

from cobra.flux_analysis.variability import flux_variability_analysis
from cobra.util.solver import linear_reaction_coefficients
from cobra.util.util import format_long_string
from cobra.core import get_solution

LOGGER = logging.getLogger(__name__)
from cobra.flux_analysis.summary import  (metabolite_summary, model_summary, _process_flux_dataframe)

# Load Various Func's

In [2]:
import FLJT

ModuleNotFoundError: No module named 'FLJT'

#  Minimal Media Characteristics

In [ ]:
# #minimal Media
# minimal_M9 = ['EX_cpd00009_LPAREN_e_RPAREN_','EX_cpd00971_LPAREN_e_RPAREN_', 'EX_cpd00067_LPAREN_e_RPAREN_',
#               'EX_cpd00001_LPAREN_e_RPAREN_',
#               'EX_cpd00205_LPAREN_e_RPAREN_', 'EX_cpd00099_LPAREN_e_RPAREN_', 'EX_cpd00013_LPAREN_e_RPAREN_',
#               'EX_cpd00528_LPAREN_e_RPAREN_', 'EX_cpd00254_LPAREN_e_RPAREN_', 'EX_cpd00048_LPAREN_e_RPAREN_', 
#               'EX_cpd00058_LPAREN_e_RPAREN_', 'EX_cpd00030_LPAREN_e_RPAREN_', 
#               'EX_cpd00034_LPAREN_e_RPAREN_', 'EX_cpd00021_LPAREN_e_RPAREN_', 'EX_cpd10516_LPAREN_e_RPAREN_',
#               'EX_cpd00007_LPAREN_e_RPAREN_',
#               'EX_cpd00011_LPAREN_e_RPAREN_', 'EX_cpd00149_LPAREN_e_RPAREN_', 'EX_cpd00254_LPAREN_e_RPAREN_',
#               'EX_cpd00244_LPAREN_e_RPAREN_',
#               'EX_cpd01012_LPAREN_e_RPAREN_'] #M9 Minimal Media components with some extra components not found in Burk Paper

# minimal_M9_2 = ['EX_cpd00007_LPAREN_e_RPAREN_','EX_cpd00011_LPAREN_e_RPAREN_','EX_cpd00067_LPAREN_e_RPAREN_', 
#                'EX_cpd00001_LPAREN_e_RPAREN_', 'EX_cpd00205_LPAREN_e_RPAREN_','EX_cpd00254_LPAREN_e_RPAREN_',
#                'EX_cpd00971_LPAREN_e_RPAREN_','EX_cpd00013_LPAREN_e_RPAREN_','EX_cpd00021_LPAREN_e_RPAREN_',
#                'EX_cpd00009_LPAREN_e_RPAREN_','EX_cpd00048_LPAREN_e_RPAREN_','EX_cpd00528_LPAREN_e_RPAREN_',
#                'EX_cpd00030_LPAREN_e_RPAREN_','EX_cpd00034_LPAREN_e_RPAREN_','EX_cpd00058_LPAREN_e_RPAREN_',
#                'EX_cpd00149_LPAREN_e_RPAREN_','EX_cpd00244_LPAREN_e_RPAREN_','EX_cpd01012_LPAREN_e_RPAREN_',
#                'EX_cpd10516_LPAREN_e_RPAREN_']# M9 Minimal Media Components in Burk Paper

# listed = model.exchanges                        #All exchange reactions 

# for i in range( len(minimal_M9_2) ):                               #for each exchange reaction in minimal_M9_2
#     if minimal_M9_2[i] in listed: listed.remove(minimal_M9_2[i])    #If Ex_rxn name in listed remove it

# #string name of multiple carbon sources + other exchanges not found in minimal media
# exchange_source_list = listed

#Notes

#minimal media (m9) is missing boron, calcium, colbalt and  molybedenum. Calcium is missing in the model. 
#also contains some elements not in atmosphere or in media, E.g. Ni, Mg or Cd
#http://jb.asm.org/content/early/2013/10/21/JB.00997-13.short ----> Minimal Media in supplementals 

# Model Copy Instantiation

In [ ]:
model = cobra.io.read_sbml_model('iPAU1129.xml', use_hyphens=False)

model_all_changes = copy.deepcopy(model)

model_all_changes_good = copy.deepcopy(model)

#Optimized Model Without ANy Changes

model.summary()


# Load model copy 

In [ ]:

#deep copy has sliiiight changes in model results 

temp = copy.deepcopy(model) #Create temporary model
# temp.optimize()


In [ ]:

#Carbon source list
carbonSource = ['EX_cpd00036_LPAREN_e_RPAREN_']                                #succinate carbon source


#contains dictionary of minimal media conditions and model optimize values

media_list = []
ind_media = []
for i in range(1, 4):#1 to number of media conditions 3 --> (4-1) = 3

    media =  FLJT.changeMedia_PA_LJD(temp, i, carbonSource)
    media.optimize()
    
    ind_media.append( media)
    ind_media.append( media.objective.value )
    media_list.append(ind_media)
    ind_media = []

for i in range(0,3):
    print('\nModel Name: ', media_list[i][0] )
    print('Model Objective Value: ', media_list[i][1]) 
    print('Model Carbon source: ', model.reactions.get_by_id(carbonSource[0]).name)


# rxn13808 load + supplementary load rxn10772


In [ ]:

rxn_name = 'rxn13808'
rxn_name_2 = 'rxn10772'    #secondary reaction with NH4OH to be analyzed for flux changes


# Results Before


In [ ]:
#flux through the two reactions in each of the media conditions

temp_flux_before = []
copy_media_temp = copy.deepcopy(media_list)

for i in range(0, 3):
        print('asdfd')
        loop_reactions = [copy_media_temp[i][0].reactions.get_by_id(rxn_name), copy_media_temp[i][0].reactions.get_by_id(rxn_name_2)]
        temp_flux_before.append( flux_variability_analysis(copy_media_temp[i][0], reaction_list=loop_reactions, loopless=True) ) #range of fluxes through the reaction
        del loop_reactions
del copy_media_temp

#Essential Genes Before - Biomass Results Before - Gene Knockouts 
copy_media_temp = copy.deepcopy(media_list)
del_results_media_after = []

for i in range(0,3):
        del_results_media_after.append( single_gene_deletion(copy_media_temp[i][0]) )
del copy_media_temp


# Reaction and Metabolite Modifications

In [ ]:

#Reaction as Written

'''
PROBLEMS: Charge imbalance +2 on the right right side 

3.0 cpd00004_c + 4.0 cpd00067_c + cpd00075_c <=> cpd00001_c + 3.0 cpd00003_c + cpd00985_c

3 NADH [0] + 4 H+ [0] + 1 Nitrite [0] -> 1 H2O [0] + 3 NAD [0] + 1 NH4OH [0]

NH4OH has charge 2 in modelseed and in model
Nh3 H2o is NH4OH... Recommended changing to neutral charge 

'''

#Reaction Changes

media_list_after = copy.deepcopy(media_list)
for i in range(0, 3):
    
    media_list_after[i][0].metabolites.get_by_id('cpd00985_c').charge = 0           #change charge to 0 
    media_list_after[i][0].optimize()
    print("Media: %s\nObjective value: %.15f" % (media_list_after[i][0], media_list_after[i][0].objective.value))


# Results Model After 

In [ ]:
#flux through the two reactions in each of the media conditions

temp_flux_after = []
copy_media_temp = copy.deepcopy(media_list_after)

for i in range(0,3):
    loop_reactions = [copy_media_temp[i][0].reactions.get_by_id(rxn_name), copy_media_temp[i][0].reactions.get_by_id(rxn_name_2)]   
    temp_flux_after.append( flux_variability_analysis(copy_media_temp[i][0], reaction_list=loop_reactions, loopless=True) ) #range of fluxes through the reaction
    del loop_reactions
del copy_media_temp

#Essential Genes Before - Biomass Results Before - Gene Knockouts 
copy_media_temp = copy.deepcopy(media_list_after)
del_results_media_after = []

for i in range(0,3):
    del_results_media_after.append( single_gene_deletion(copy_media_temp[i][0]) )
del copy_media_temp

# Check if FreeMass Exist

In [ ]:
for i in range(0,3):
    print('Results from free mass are: ',FLJT.checkFreeMass(media_list_after[i][0], cytosol='c'))

# Flux, objective values and deletion results printed

In [ ]:
#difference in objective values
# print('Objective value original - objective value after changes\n')
# for i in range(0, 3):
#     print("Media: %s\nObjective value change: %.20f" % (media_list_after[i][0], media_list_after[i][0].objective.value))

    #fluxes before and after
print('\nFlux results are:\n\n')
print(temp_flux_before)
print(temp_flux_after)    
type(temp_flux_after)
# print('\nDeletion results are results are:\n\n')
# print(del_results_media_after)
# print(del_results_media_before)


# Comment on results


NH4OH is +2 charge in the model and in modelseed github. 
Changing the charge doubles the biomass. 
This appears to 


In [ ]:
FLJT.minimal_media_varied_carbon(media_list_after[0][0])